In [ ]:
file_path = 'HPG Historical Data 1.csv'  # Replace with the correct path
data = pd.read_csv(file_path)
data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,03/31/2023,"18,909.0","18,909.0","19,000.0","18,500.0",26.15M,0.00%
1,03/30/2023,"18,909.0","19,091.0","19,227.0","18,864.0",23.05M,0.00%
2,03/29/2023,"18,909.0","18,818.0","19,182.0","18,818.0",19.74M,0.48%
3,03/28/2023,"18,818.0","19,000.0","19,091.0","18,818.0",13.56M,-0.48%
4,03/27/2023,"18,909.0","18,545.0","19,000.0","18,500.0",22.29M,1.96%


In [ ]:
data['Price'] = data['Price'].str.replace(',','')
data['Open'] = data['Open'].str.replace(',','')
data['High'] = data['High'].str.replace(',','')
data['Low'] = data['Low'].str.replace(',','')
data['Vol.'] = data['Vol.'].str.replace('.','')
data['Vol.'] = data['Vol.'].str.replace('M','')
data['Change %'] = data['Change %'].str.replace('%','')
data.head()


,Date,Price,Open,High,Low,Vol.,Change %
0,03/31/2023,18909.0,18909.0,19000.0,18500.0,2615,0.00
1,03/30/2023,18909.0,19091.0,19227.0,18864.0,2305,0.00
2,03/29/2023,18909.0,18818.0,19182.0,18818.0,1974,0.48
3,03/28/2023,18818.0,19000.0,19091.0,18818.0,1356,-0.48
4,03/27/2023,18909.0,18545.0,19000.0,18500.0,2229,1.96


In [ ]:
data[ "Price"] = data[ "Price"].astype("float")
data[ "Open"] = data[ "Open"].astype("float")
data[ "High"] = data[ "High"].astype("float")
data[ "Low"] = data[ "Low"].astype("float")
data[ "Vol."] = data[ "Vol."].astype("float")
data[ "Change %"] = data[ "Change %"].astype("float")


In [ ]:

data[ "Vol."] = data[ "Vol."]*10000
data[ "Change %"] = data[ "Change %"]/100


In [ ]:
data

,Date,Price,Open,High,Low,Vol.,Change %
0,03/31/2023,18909.0,18909.0,19000.0,18500.0,26150000.0,0.0000
1,03/30/2023,18909.0,19091.0,19227.0,18864.0,23050000.0,0.0000
2,03/29/2023,18909.0,18818.0,19182.0,18818.0,19740000.0,0.0048
3,03/28/2023,18818.0,19000.0,19091.0,18818.0,13560000.0,-0.0048
4,03/27/2023,18909.0,18545.0,19000.0,18500.0,22290000.0,0.0196
...,...,...,...,...,...,...,...
1303,01/08/2018,12443.5,11991.0,12443.5,11940.7,14190000.0,0.0313
1304,01/05/2018,12066.4,12091.6,12179.6,11978.4,14030000.0,-0.0042
1305,01/04/2018,12116.7,12016.2,12167.0,11965.9,17790000.0,0.0126
1306,01/03/2018,11965.9,12066.4,12192.1,11890.5,18510000.0,-0.0021


In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data.sort_index(inplace=True)

In [ ]:
data

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2018-01-02,11991.0,11789.9,12003.6,11676.8,15730000.0,0.0181
2018-01-03,11965.9,12066.4,12192.1,11890.5,18510000.0,-0.0021
2018-01-04,12116.7,12016.2,12167.0,11965.9,17790000.0,0.0126
2018-01-05,12066.4,12091.6,12179.6,11978.4,14030000.0,-0.0042
2018-01-08,12443.5,11991.0,12443.5,11940.7,14190000.0,0.0313
...,...,...,...,...,...,...
2023-03-27,18909.0,18545.0,19000.0,18500.0,22290000.0,0.0196
2023-03-28,18818.0,19000.0,19091.0,18818.0,13560000.0,-0.0048
2023-03-29,18909.0,18818.0,19182.0,18818.0,19740000.0,0.0048


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Select relevant columns (assuming 'Close' is the target variable)
prices = data['Price'].values.reshape(-1, 1)


# Min-Max Scaling
scaler = MinMaxScaler()
prices_scaled = scaler.fit_transform(prices)

# Standardization
standardizer = StandardScaler()
prices_standardized = standardizer.fit_transform(prices_scaled)

# Split the data into training and testing sets based on dates
train_data = data.loc['2018-01-02':'2022-02-28']
test_data = data.loc['2022-03-01':'2023-03-31']

# Apply the same transformations to the split data
train_prices = prices_standardized[:len(train_data)]
test_prices = prices_standardized[len(train_data):len(train_data) + len(test_data)]

# Function to create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 60  # Example sequence length
X_train, y_train = create_sequences(train_prices, seq_length)
X_test, y_test = create_sequences(test_prices, seq_length)

# Ensure the data is in the correct shape for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


# Build the LSTM model
def build_model(optimizer='adam', units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(seq_length, 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Manual hyperparameter tuning
best_score = float('inf')
best_params = {}

# Example of hyperparameters to try
optimizers = ['adam', 'rmsprop']
units = [50, 100, 150]
dropout_rates = [0.2, 0.3, 0.4]
batch_sizes = [16, 32, 64]
epochs = [10, 20, 30]

for optimizer in optimizers:
    for unit in units:
        for dropout_rate in dropout_rates:
            for batch_size in batch_sizes:
                for epoch in epochs:
                    model = build_model(optimizer=optimizer, units=unit, dropout_rate=dropout_rate)
                    model.fit(X_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)
                    y_pred = model.predict(X_test)

                    # Rescale predictions
                    y_pred_rescaled = standardizer.inverse_transform(y_pred.reshape(-1, 1))
                    y_test_rescaled = standardizer.inverse_transform(y_test.reshape(-1, 1))

                    # Metrics
                    rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))

                    if rmse < best_score:
                        best_score = rmse
                        best_params = {
                            'optimizer': optimizer,
                            'units': unit,
                            'dropout_rate': dropout_rate,
                            'batch_size': batch_size,
                            'epochs': epoch
                        }

print(f'Best parameters: {best_params}')
print(f'Best RMSE: {best_score}')



C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
Best parameters: {'optimizer': 'rmsprop', 'units': 150, 'dropout_rate': 0.3, 'batch_size': 16, 'epochs': 30}
Best RMSE: 0.018781533434571018


TypeError: build_model() got an unexpected keyword argument 'batch_size'

In [ ]:
# Train final model with best parameters
from tensorflow.keras.optimizers import RMSprop
def build_best_model():
    model = Sequential()
    model.add(LSTM(units=150, return_sequences=True, input_shape=(seq_length, 1)))
    model.add(Dropout(0.3))
    model.add(LSTM(units=150))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer=RMSprop(), loss='mean_squared_error')
    return model

# Build the model with best parameters
best_model = build_best_model()

# Train the model
best_model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1)

# Evaluate the model
y_pred = best_model.predict(X_test)
y_pred_rescaled = standardizer.inverse_transform(y_pred.reshape(-1, 1))
y_test_rescaled = standardizer.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_rescaled, y_pred_rescaled)
mape = mean_absolute_percentage_error(y_test_rescaled, y_pred_rescaled)
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))

print(f'R square: {r2}')
print(f'MAPE: {mape}')
print(f'RMSE: {rmse}')


Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 0.2014
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0366
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0267
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0255
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0228
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0222
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0203
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0183
Epoch 9/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0196
Epoch 10/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0193
Epoch 11/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0150
Epoch 12/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0163
Epoch 13/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0218
Epoch 14/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0160
Epoch 15/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0166
Epoc